# 3-Stage Clos Network to make Networks Scalable
###### <sup>Inputs: Bhavani Parise, Deepti Chandra; Developed by: Sarah Samuel</sup>
###### <sup>Read use case on Devnet: https://developer.cisco.com/docs/sonic/#!3-stage-clos-network-with-static-vxlan-to-make-networks-scalable</sup>

Modern cloud-scale data center networks require increased server-to-server communication over a network that stays resilient despite the rapid increase in the number of devices.

The 3-stage Clos network is a robust IP-BGP underlay network that enables the servers to communicate with each other with minimum latency. Over this network, you can configure overlay features such as Virtual Extensible Local Area Network (VXLAN).

A 3-stage Clos network consists of data center network switches where each spine switch connects to all leaf switches. And each leaf switch connects to a rack of servers in the data center. Any server in the data center is just three hops away from another server. The first hop is from the server to the directly connected leaf switch, the second hop is across the spine switches to the destination leaf switch, and the third hop is between the destination leaf switch to the destination server. This network architecture is highly scalable. Also, irrespective of the number of devices in the data center, the number of hops between the servers or the end-hosts is always 3, ensuring consistent latency in the data center network..

Though you use generally switches in a 3-stage Clos network, this notebook demonstrates how to [configure 3-Stage Clos Network](#configure-3-stage-clos-network) using Cisco 8000 series routers that run SONiC.

The following topology diagram depicts a simple 3-stage Clos network with two leaf routers in Tier-0 and two spine routers in Tier-1.



<center><img src="./images/522559.jpg" width="700"/></center>


As you read through this notebook, play the code-cells, using the play button in the top left corner of this page, to send configuration commands to the simulated devices that are connected as shown in the topology diagram. The notebook refreshes the output of each cell-execution just beneath it.

The devices in the topology are brought up when you reserved the sandbox. Now you can play the code-cells to [access device consoles](#Access-Device-Consoles) and continue playing through the following steps to [configure the 3-stage Clos network](#Configure-3-Stage-Clos-Network):
* [Configure Host-Names](#Configure-Host-Names)
* [Assign IP-Addresses](#Assign-IP-Addresses)
* [Configure eBGP](#Configure-eBGP)
* [Verify BGP Route Exchange](#Verify-BGP-Route-Exchange)
* [Send Traffic from TREX](#Send-Traffic-from-TREX)
* [Verify Traffic Statistics](#Verify-Traffic-Statistics)

### Access Device Consoles

> Play the following cell to access the SSH console of each device in the topology. 



In [ ]:
from lib.leaf_spine import *
nodes = {
         'S0':'', 
         'S1':'',
         'L0':'', 
         'L1':'', 
         'trex':''
        }

tb = access_device_consoles("lib/leaf_spine.yaml", nodes)

## Configure 3-Stage Clos Network

> To configure a 3-stage clos network, continue playing through the following steps:

### Configure Host-Names

> Configure host names for the spine and leaf routers for easy identification.

In [ ]:
out = nodes['S0'].execute('sudo config hostname SPINE0')
out = nodes['S0'].execute('sudo config save -y')
out = nodes['S1'].execute('sudo config hostname SPINE1')
out = nodes['S1'].execute('sudo config save -y')
out = nodes['L0'].execute('sudo config hostname LEAF0')
out = nodes['L0'].execute('sudo config save -y')
out = nodes['L1'].execute('sudo config hostname LEAF1')
out = nodes['L1'].execute('sudo config save -y')

### Assign IP Addresses

> This step assigns IP addresses on the interfaces of all devices as per the topology diagram. And then saves the configurations.

In [ ]:
print ("\n******************** Configuring S0 *************************")
out = nodes['S0'].execute('sudo config interface ip add Ethernet0 10.0.1.1/24')
out = nodes['S0'].execute('sudo config interface ip add Ethernet8 10.0.2.1/24')
out = nodes['S0'].execute('sudo config interface ip add Loopback0 10.10.10.100/32')
out = nodes['S0'].execute('sudo config save -y')
print ("\n******************** Configuring S1 *************************")
out = nodes['S1'].execute('sudo config interface ip add Ethernet0 10.0.3.1/24')
out = nodes['S1'].execute('sudo config interface ip add Ethernet8 10.0.4.1/24')
out = nodes['S1'].execute('sudo config interface ip add Loopback0 10.10.11.100/32')
out = nodes['S1'].execute('sudo config save -y')
print ("\n******************** Configuring L0 *************************")
out = nodes['L0'].execute('sudo config interface ip add Ethernet0 10.0.1.2/24')
out = nodes['L0'].execute('sudo config interface ip add Ethernet12 10.0.3.2/24')
out = nodes['L0'].execute('sudo config interface ip add Ethernet8 10.0.5.1/24')
out = nodes['L0'].execute('sudo config interface ip add Loopback0 10.10.10.200/32')
out = nodes['L0'].execute('sudo config save -y')
print ("\n******************** Configuring L1 *************************")
out = nodes['L1'].execute('sudo config interface ip add Ethernet12 10.0.2.2/24')
out = nodes['L1'].execute('sudo config interface ip add Ethernet0 10.0.4.2/24')
out = nodes['L1'].execute('sudo config interface ip add Ethernet8 10.0.6.1/24')
out = nodes['L1'].execute('sudo config interface ip add Loopback0 10.10.11.200/32')
out = nodes['L1'].execute('sudo config save -y')
print ("\n******************** Configuring TREX *************************")
out = nodes['trex'].execute('ifconfig eth1 10.0.5.2 netmask 255.255.255.0 up')
out = nodes['trex'].execute('ifconfig eth2 10.0.6.2 netmask 255.255.255.0 up')

> Verify the configured IP addresses using the ```show ip interfaces``` command on the routers and ```ifconfig -a``` command on the TREX server.

In [ ]:
for n in nodes:
   print ("\n******************** Command Outputs from", n, "*************************")
   if (n == 'trex'):
      out = nodes[n].execute('ifconfig -a eth1')
      out = nodes[n].execute('ifconfig -a eth2')
   else:
      out = nodes[n].execute('show ip interfaces')  

### Configure eBGP

> The following section shows the eBGP (exterior Border Gateway Protocol) configuration on all spine and leaf routers. The spines are in Autonomous System (AS) 100 and the leaves in AS 200. 
> Note the usage of the ```allowas-in``` option in the ```neighbor``` command. The default behaviour of BGP is to reject learning routes from own AS. The ```allowas-in``` option overrides this default behavior and the nodes learn routes from own AS. 

In [ ]:
print ("\n******************** Configuring eBGP on S0 *************************")
out = nodes['S0'].execute ('''vtysh \
-c 'configure terminal' \
-c 'hostname SPINE_0' \
-c 'router-id 10.10.10.100' \
-c 'router bgp 100' \
-c 'no bgp ebgp-requires-policy' \
-c 'neighbor 10.0.2.2 remote-as 200' \
-c 'neighbor 10.0.1.2 remote-as 200' \
-c 'address-family ipv4 unicast' \
-c 'neighbor 10.0.2.2 allowas-in' \
-c 'neighbor 10.0.1.2 allowas-in' \
-c 'network 10.0.1.0/24' \
-c 'network 10.0.2.0/24' \
-c 'network 10.10.10.100/32' \
-c 'redistribute connected'
''')
print ("\n******************** Configuring eBGP on S1 *************************")
out = nodes['S1'].execute ('''vtysh \
-c 'configure terminal' \
-c 'hostname SPINE_1' \
-c 'router-id 10.10.11.100' \
-c 'router bgp 100' \
-c 'no bgp ebgp-requires-policy' \
-c 'neighbor 10.0.3.2 remote-as 200' \
-c 'neighbor 10.0.4.2 remote-as 200' \
-c 'address-family ipv4 unicast' \
-c 'neighbor 10.0.3.2 allowas-in' \
-c 'neighbor 10.0.4.2 allowas-in' \
-c 'network 10.0.3.0/24' \
-c 'network 10.0.4.0/24' \
-c 'network 10.10.11.100/32' \
-c 'redistribute connected'
''')
print ("\n******************** Configuring eBGP on L0 *************************")
out = nodes['L0'].execute ('''vtysh \
-c 'configure terminal' \
-c 'hostname LEAF_0' \
-c 'router-id 10.10.10.200' \
-c 'router bgp 200' \
-c 'no bgp ebgp-requires-policy' \
-c 'neighbor 10.0.1.1 remote-as 100' \
-c 'neighbor 10.0.3.1 remote-as 100' \
-c 'address-family ipv4 unicast' \
-c 'neighbor 10.0.1.1 allowas-in' \
-c 'neighbor 10.0.3.1 allowas-in' \
-c 'network 10.0.1.0/24' \
-c 'network 10.0.3.0/24' \
-c 'network 10.0.5.0/24' \
-c 'network 10.10.10.200/32' \
-c 'redistribute connected'
''')
print ("\n******************** Configuring eBGP on L1 *************************")
out = nodes['L1'].execute ('''vtysh \
-c 'configure terminal' \
-c 'hostname LEAF_1' \
-c 'router-id 10.10.11.200' \
-c 'router bgp 200' \
-c 'no bgp ebgp-requires-policy' \
-c 'neighbor 10.0.2.1 remote-as 100' \
-c 'neighbor 10.0.4.1 remote-as 100' \
-c 'address-family ipv4 unicast' \
-c 'neighbor 10.0.2.1 allowas-in' \
-c 'neighbor 10.0.4.1 allowas-in' \
-c 'network 10.0.2.0/24' \
-c 'network 10.0.4.0/24' \
-c 'network 10.0.6.0/24' \
-c 'network 10.10.11.200/32' \
-c 'redistribute connected'
''')
time.sleep(60)

> Execute the command ```show ip bgp``` on all tier-0 and tier-1 routers to ensure eBGP is up.

In [ ]:
for n in nodes:
   if (n != 'trex'):
      print ("\n******************** Command Outputs from", n, "*************************")
      out = nodes[n].execute('''vtysh \
      -c 'show ip bgp'
      ''')

> This section checks the routes learnt through BGP by executing the command ```show ip route``` on all nodes in tier-0 and tier-1. The lines starting with the letter **B** are the routes learnt through BGP.

In [ ]:
for n in nodes:
   if (n != 'trex'):
      print ("\n******************** Command Outputs from", n, "*************************")
      out = nodes[n].execute('''vtysh \
      -c 'show ip route'
      ''')

> Ping IP addresses on the LEAF0 (L0) router from the LEAF1 (L1) router and vice-versa. This ensures that the 3-stage Clos network is ready for end-to-end traffic flow.

In [ ]:

print ("\n******************** Ping from L1 to L0 *************************")
out = nodes['L1'].execute('ping -c5 10.10.10.200')
print ("\n******************** Ping from L0 to L1 *************************")
out = nodes['L0'].execute('ping -c5 10.10.11.200')

**Pinging to remote leaf ip address is successful and so the 3-stage Clos network is up and ready for data traffic**

### Send Traffic from TREX

> [TREX](https://trex-tgn.cisco.com/trex/doc/trex_manual.html#_introduction) is a software traffic generator that runs on Linux. To simulate server-to-server traffic flow across a 3-stage Clos network, connect the TREX software traffic generator ports as the end-hosts to LEAF0 and LEAF1. 

> Details of traffic stream injected into LEAF0 from TREX:
> * Source IP address: 10.0.5.2
> * Destination IP address: 10.0.6.2

> Details of traffic stream injected into LEAF1 from TREX:
> * Source IP address: 10.0.6.2
> * Destination IP address: 10.0.5.2

<center><img src="./images/3clos-traffic.png" width="700"/></center>

> The function ```generate_bidir_traffic``` generates a bidirectional traffic burst for 1 second. After the cell execution, check ```Total-tx-pkt``` and ```Total-rx-pkt``` in the ```summary stats``` at the end of the output to ensure that there is no traffic loss.

In [ ]:
trex_ipaddress = str(nodes['trex'].connections.cli.ip)
trex_port = str(nodes['trex'].connections.cli.port)

generate_bidir_traffic(trex_ipaddress, trex_port)

### Verify Traffic Statistics

> Check the interface counters on the nodes to ensure that the traffic is sent and received as seen in the traffic generator output.
    
> Note: "show interface counters rif" displaying 0s for all fields, is a known issue on the emulator. Users will experience expected behaviours on the physical setup.


In [ ]:
for n in nodes:
   if (n != 'trex'):
      out = nodes[n].execute('show interface counters rif')

> You have now successfully brought up a simple 3-stage Clos network, sent traffic across it. This forms the IP-BGP underlay for your data center network. 

> Scale up these configurations as per the number of devices in your data centre. 

### Clean up Router Configurations

> Remove the router configurations so that you can play other notebooks

In [ ]:
# Remove the Router configs and release the interfaces used by trex
out = nodes['S0'].execute('sudo config interface ip remove Ethernet0 10.0.1.1/24')
out = nodes['S0'].execute('sudo config interface ip remove Ethernet8 10.0.2.1/24')
out = nodes['S0'].execute('sudo config interface ip remove Loopback0 10.10.10.100/32')
out = nodes['S1'].execute('sudo config interface ip remove Ethernet0 10.0.3.1/24')
out = nodes['S1'].execute('sudo config interface ip remove Ethernet8 10.0.4.1/24')
out = nodes['S1'].execute('sudo config interface ip remove Loopback0 10.10.11.100/32')
out = nodes['L0'].execute('sudo config interface ip remove Ethernet0 10.0.1.2/24')
out = nodes['L0'].execute('sudo config interface ip remove Ethernet12 10.0.3.2/24')
out = nodes['L0'].execute('sudo config interface ip remove Ethernet8 10.0.5.1/24')
out = nodes['L0'].execute('sudo config interface ip remove Loopback0 10.10.10.200/32')
out = nodes['L1'].execute('sudo config interface ip remove Ethernet12 10.0.2.2/24')
out = nodes['L1'].execute('sudo config interface ip remove Ethernet0 10.0.4.2/24')
out = nodes['L1'].execute('sudo config interface ip remove Ethernet8 10.0.6.1/24')
out = nodes['L1'].execute('sudo config interface ip remove Loopback0 10.10.11.200/32')

for n in nodes:
   if (n != 'trex'):
      print("cfg cleaned")
      out = nodes[n].execute('sudo rm /etc/sonic/config_db.json')
   else:
      out = nodes[n].execute('cd /opt/cisco/trex/latest/')
      out = nodes[n].execute('sudo ./dpdk_nic_bind.py --force -u 00:04.0')
      out = nodes[n].execute('sudo ./dpdk_nic_bind.py --force -u 00:05.0')
      out = nodes[n].execute('sudo ./dpdk_nic_bind.py --bind=virtio-pci 00:04.0')
      out = nodes[n].execute('sudo ./dpdk_nic_bind.py --bind=virtio-pci 00:05.0')

> Do let us know of your feedback or queries about this notebook at mig-notebooks@cisco.com.